In [1]:
!pip install pandas
!pip install music21


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import pickle
import pandas as pd
import music21
from datetime import datetime, timedelta

In [11]:
piece_id_counter = 1
phrase_id_counter = 1

In [12]:
data = []

files = ["data_alsace"]
for file in files:
    with open('../data/data_2/{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

In [19]:
for piece in data:
    notes = piece[0]
    diff = None
    reset = True 
    for i, p in enumerate(piece[2]):
        if reset:
            diff = music21.note.Note("C4").pitch.midi - notes[i]
            reset = False
        piece[0][i] += diff
        if p == 1:
            reset = True

In [17]:
from datetime import datetime, timedelta
import pandas as pd
import music21

df = pd.DataFrame(columns=['PhraseID', 'Note', 'TimeStamp', 'PieceID', 'start_timestamp', 'end_timestamp'])
piece_id_counter = 1
phrase_id_counter = 1
start_time = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

for piece in data:
    piece_id = "PIECE{:05}".format(piece_id_counter)
    timestamp = start_time
    start_timestamp = timestamp

    new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': 'START',
                                  'PieceID': [piece_id], 'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                  'end_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")]})
    df = pd.concat([df, new_item], ignore_index=True)

    for i in range(len(piece[0])):
        if piece[0][i] == 128:
            note = "rest"
        else:
            note_with_octave = music21.note.Note(piece[0][i]).nameWithOctave
            note = note_with_octave[:-1]  # Usuwanie ostatniego znaku (oktawy)

        duration = piece[1][i]
        phrase_id = "PHRASE{:06}".format(phrase_id_counter)
        timestamp += timedelta(seconds=int(10 * duration))

        new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': [note],
                                      'TimeStamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                      "PieceID": [piece_id],
                                      'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                      'end_timestamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")]})
        df = pd.concat([df, new_item], ignore_index=True)
        start_timestamp = timestamp
        if piece[2][i] == 1:
            timestamp = start_time
            phrase_id_counter += 1
            new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': 'END',
                                           'TimeStamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                           "PieceID": [piece_id],
                                           'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                           'end_timestamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")]})
            df = pd.concat([df, new_item], ignore_index=True)
            start_timestamp = start_time
            phrase_id = "PHRASE{:06}".format(phrase_id_counter)
            new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': 'START',
                                'PieceID': [piece_id], 'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                'end_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")]})
            df = pd.concat([df, new_item], ignore_index=True)

    piece_id_counter += 1
    if piece_id_counter % 1000 == 0:
        df.to_csv("log_data{}.csv".format(piece_id_counter))
        df = pd.DataFrame(columns=['PhraseID', 'Note', 'TimeStamp', 'PieceID', 'start_timestamp', 'end_timestamp'])
    print(piece_id_counter)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


In [18]:
df.to_csv("log_data.csv")